**1.) Importing the libraries**

In [41]:
import numpy as np
from sklearn.datasets import load_iris
data = load_iris()
data.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

**2.) DATA SPLITING**
   we are taking last two columns of X from iris dataset
   for y classifying virginica from others



In [19]:
X = data['data'][:,2:]
y = (data["target"] == 2).astype(np.float64).reshape(-1,1)


**3.)Algorithm**



1.   L = |w|^2 /2 - sum(c[(w.X+b)* y -1])
2.   differentiate L with respect to w =>  w - sum(C* X * y) 
3.   differentiate L with respect to b => - sum(C * y) 
4. for both cases the (w.X+b)* y < 1  must be true because we should classify    only the mi-sclassified data points.

In [40]:
class RETSVM:
  def __init__(self, epoches=60000, C=2, lr = 10 ,random_state = 2):
    self.epoches = epoches
    self.C = C
    self.lr = lr
    self.random_state = random_state

  def fit(self,X,y):    
    self.X = X    
    self.y = y

  def lrtrimmer(self,count):
    return self.lr / (count + 1000)

  def train(self): 
    if self.random_state:
      np.random.seed(self.random_state)
    w = np.array([[0],[0]])
    b = 0
    
    y = self.y*2 - 1  # changing -1 for 0's and 1 for 1's 
    X =  self.X

    for ep in range(self.epoches):        
      to_change_points = (np.dot(y*X , w) + y * b < 1).ravel() # splitting the indexes of the points  which are misclassified
      X_toChange = X[to_change_points] # applying those indexes to the X and Y values for filtering
      y_toChange = y[to_change_points]


      w_derivative = w - self.C*np.sum(X_toChange*y_toChange,axis=0).reshape(-1,1) # calculating derivatives for respect to w,b
      b_derivative = -self.C*np.sum(y_toChange)
        
      w = w - self.lrtrimmer(ep)*w_derivative # learning rate is minimized for every itreations 
      b = b - self.lrtrimmer(ep)*b_derivative

    self.coefficients = w
    self.intercept = b
    return self

  def predict(self,X):
    return (X.dot(self.coefficients) + self.intercept > 0).astype(np.float64)
     
my_svm_clf = RETSVM(epoches = 100000 ,C=1,lr=10,random_state=2)
my_svm_clf.fit(X,y)
my_svm_clf.train()


print(my_svm_clf.intercept , my_svm_clf.coefficients)



-14.360737837504225 [[2.17092238]
 [2.25889228]]


**4**.) **Comparision** 

  both RETSVM and SVC calculated similar results.

In [38]:
from sklearn.svm import SVC

clf = SVC(kernel="linear" ,C=1)
clf.fit(X,y)

print(clf.intercept_ , clf.coef_)

[-14.41486828] [[2.1829247  2.25365588]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**5.) Thanks to**


*   Alexandre KOWALCZYK(https://www.svm-tutorial.com/)
*   Aurelie Geron (hands on machine learning with scikit-learn and tensorflow)

  for teaching me the concepts

